In [ ]:
#동물병원 CSV 원본파일 PYSPARK를 통한 전처리작업
from pyspark.sql import *
from pyspark.sql.functions import col, lit
import pyspark.sql.functions 


medical_read=spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("encoding","cp949").load('/user/medical.csv/medicaladdress.csv')

sort1=medical_read.select(col("사업장명"),col("소재지전체주소"),col("소재지전화"),col("좌표정보(x)"),col("좌표정보(y)"))
#sort1 /9589
sort1.show()

#비어있는 값이 있으면 무조건 해당 로우삭제.
# drop_null /6402
drop_null=sort1.na.drop("any")
drop_null.show(6403)

#약국과 병원 분리작업
#해당 단어포함시 총갯수 기록:병원만=5665/메디컬 포함=5845 /센터포함=6194

medical_filter=drop_null.filter(col('사업장명').like('%병원%') | col('사업장명').like('%메디컬%') | col('사업장명').like('%센터%'))
medical_filter.show(6197)

#medical_final_1.csv생성
medical_filter.coalesce(1).write.options(header='True', delimiter=',', encoding="cp949").csv('/user/medical/medical_final_1.csv/')

In [ ]:
# Pyspark로 전처리할때 빠트린 중복값 검색,제거, 중복제거로일어난 id값 재정렬 

#id제외 위도,경도 중복값 조회
SELECT lot,lat, COUNT(*) as cnt
FROM medical #위에서 만든 medical이라는 테이블
GROUP BY lot,lat
HAVING COUNT(lot) > 1 AND COUNT(lat) > 1;

#위의 셀렉트결과를 토대로 중복값 제거
DELETE t1 FROM medical t1 
JOIN medical t2
ON t1.lot=t2.lot AND t1.lat=t2.lat
WHERE t1.id > t2.id;

#삭제로 일정하지않은 id를 재생성
SET @CON =0;
UPDATE medical SET id = @CON:=@CON+1;
